In [26]:
import mujoco
import mujoco.viewer
import numpy as np
import time
import scipy.optimize
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import csv
from plotly.subplots import make_subplots

In [67]:
class MuscleModel:
    def __init__(self, scale: float, wrist_on: bool = False, model_path: str = "model.xml", emg_muscles = []):
        self.scale = scale
        self.wrist_on = wrist_on
        self.model_path = model_path

        if self.wrist_on:
            self.muscle_names = []
            self.joint_names = []  
        else:
            # self.muscle_names = [
            #     "DELT1", "DELT2", "DELT3", "SUPSP", "INFSP", "SUBSC", "TMIN", "TMAJ",
            #     "PECM1", "PECM2", "PECM3", "LAT1", "LAT2", "LAT3", "CORB", "TRIlong",
            #     "TRIlat", "TRImed", "ANC", "SUP", "BIClong", "BICshort", "BRA", "BRD"
            # ]

            # LAT1 not working for some reason

            self.muscle_names = [
                "DELT1", "DELT2", "DELT3", "SUPSP", "INFSP", "SUBSC", "TMIN", "TMAJ",
                "PECM1", "PECM2", "PECM3", "LAT2", "LAT3", "CORB", "TRIlong",
                "TRIlat", "TRImed", "ANC", "SUP", "BIClong", "BICshort", "BRA", "BRD"
            ]

            self.joint_names = [
                # 'acromioclavicular_r1', 'acromioclavicular_r2', 'acromioclavicular_r3', 
                'elv_angle',
                'shoulder_elv',
                'shoulder_rot', 
                'elbow_flexion',
                # 'pro_sup', 
                # 'flexion', 'deviation', 
                # 'shoulder1_r2',
                # 'sternoclavicular_r2', 'sternoclavicular_r3',
                # 'unrothum_r1', 'unrothum_r2', 'unrothum_r3', 'unrotscap_r2', 'unrotscap_r3'
            ]

        self.model = mujoco.MjModel.from_xml_path(self.model_path)
        self.data = mujoco.MjData(self.model)
        # mujoco.mj_forward(self.model, self.data)
        # self._scale_body_mass() 

        self.muscle_ids = [mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, name) for name in self.muscle_names]
        
        self.act = np.zeros_like(self.data.act)
        
        self.act[self.muscle_ids] = 0.1

        self.elv_angle_id = self.model.joint('elv_angle').dofadr
        self.shoulder_elv_id = self.model.joint('shoulder_elv').dofadr
        self.rot_id = self.model.joint('shoulder_rot').dofadr
        self.elbow_id = self.model.joint('elbow_flexion').dofadr

        self.joint_id_list = [self.elv_angle_id, self.shoulder_elv_id, self.rot_id, self.elbow_id]

        self.model.opt.timestep = 0.005

    # def _scale_body_mass(self):
    #     try:
    #         body_id = self.model.body(self.target_body)
    #         original_mass = self.model.body_mass[body_id]
    #         self.model.body_mass[body_id] = original_mass * self.scale
    #         print(f"Scaled mass of '{self.target_body}' from {original_mass} to {self.model.body_mass[body_id]}")
    #     except Exception as e:
    #         print(f"Error scaling body mass: {e}")

    def optimize(self, req_torques: list, emg_act = []):
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        state0 = np.empty(size, np.float64)
        mujoco.mj_getState(self.model, self.data, state0, spec)

        #assuming act contains all the correct activations from previous instant, enforce emg based activations
        for i in range(len(emg_act)):
            self.act[self.emg_muscle_ids[i]] = emg_act[i]
        
        initial_guess = [self.act[i] for i in self.muscle_ids] #set initial guess to activations from previous step
        
        def cost_fn(activations):
            return np.sum(activations**2)

        def torque_constraint(activations):
            
            mujoco.mj_setState(self.model, self.data, state0, spec)
            # mujoco.mj_step(self.model, self.data)

            # for i in range(len(self.joint_names)):
            #     self.data.qpos[mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_JOINT, self.joint_names[i])] = qpos[i]
            #     self.data.qvel[mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_JOINT, self.joint_names[i])] = qvel[i]
            
            ctrl = self.data.ctrl
            
            for idx, act in zip(self.muscle_ids, activations):
                ctrl[idx] = act

            self.data.ctrl = ctrl
            for _ in range(2):
                self.data.ctrl = ctrl
                mujoco.mj_step(self.model, self.data)
            
            elv_angle_torq = self.data.qfrc_actuator[self.elv_angle_id]
            shoulder_elv_torq = self.data.qfrc_actuator[self.shoulder_elv_id]
            shoulder_rot_torq = self.data.qfrc_actuator[self.rot_id]
            elbow_torq = self.data.qfrc_actuator[self.elbow_id]

            torq = [elv_angle_torq[0], shoulder_elv_torq[0], shoulder_rot_torq[0], elbow_torq[0]]
            # print(activations)
            # print(torq)
            return req_torques - torq

        constraints = {
            'type': 'eq',
            'fun': torque_constraint,
            'tol': 1e-3
        }

        bounds = [(0, 1)] * len(self.muscle_ids)

        start_t = time.time_ns()
        res = scipy.optimize.minimize(
            fun=cost_fn,
            x0=initial_guess,
            method='SLSQP',
            constraints=[constraints],
            bounds=bounds,
            options={'disp': False, 'maxiter': 100}
        )
        end_t = time.time_ns()
        print(f"Optimization took {(end_t - start_t) / 1e6} ms")

        if not res.success:
            print("Optimization failed:", res.message)

        elv_angle_torq = self.data.qfrc_actuator[self.elv_angle_id] + self.data.qfrc_passive[self.elv_angle_id] + self.data.qfrc_applied[self.elv_angle_id]
        shoulder_elv_torq = self.data.qfrc_actuator[self.shoulder_elv_id] + self.data.qfrc_passive[self.shoulder_elv_id] + self.data.qfrc_applied[self.shoulder_elv_id]
        shoulder_rot_torq = self.data.qfrc_actuator[self.rot_id] + self.data.qfrc_passive[self.rot_id] + self.data.qfrc_applied[self.rot_id]
        elbow_torq = self.data.qfrc_actuator[self.elbow_id] + self.data.qfrc_passive[self.elbow_id] + self.data.qfrc_applied[self.elbow_id]

        torq = [elv_angle_torq[0], shoulder_elv_torq[0], shoulder_rot_torq[0], elbow_torq[0]]

        return res.x, res.fun, res.success, end_t - start_t, torq
    
    def stiffness(self):
        K_muscle = []
        # alpha = 23.4
        alpha = 23.4
        muscle_forces = []

        for id in self.muscle_ids:
            f = - self.data.actuator_force[id]
            l = self.data.actuator_length[id]
            km = alpha * f/l
            K_muscle.append(km)
            muscle_forces.append(f)
        
        muscle_forces = np.array(muscle_forces)

        jac = self.data.ten_J
        jac_1 = []
        
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        state1 = np.empty(size, np.float64)
        mujoco.mj_getState(model.model, model.data, state1, spec)

        for i in range(self.model.na):
            if i in self.muscle_ids:
                jac_1.append([])
                for j in range(self.model.nq):
                    if j in self.joint_id_list:
                        jac_1[-1].append(jac[i+4, j])

        jac_1 = np.array(jac_1)
        dJdT = []
        print(jac_1)

        for id in self.joint_id_list:
            jac_t1 = []
            dJ = []
            mujoco.mj_setState(model.model, model.data, state1, spec)
            self.data.qpos[id] += 0.001
            mujoco.mj_forward(self.model, self.data)
            jac = self.data.ten_J

            for i in range(self.model.na):
                if i in self.muscle_ids:
                    jac_t1.append([])
                    for j in range(self.model.nq):
                        if j in self.joint_id_list:
                            jac_t1[-1].append(jac[i+4, j])
            
            jac_t1 = np.array(jac_t1)
        
            dJ = (jac_t1.T - jac_1.T)/0.001

            dJdT.append(dJ)

        dJdT = np.array(dJdT)
        k_diffterm = dJdT @ muscle_forces
        # k_diffterm_col = dJ @ muscle_forces
        # k_diffterm = [k_diffterm_col for _ in range(len(self.joint_id_list))]
        # k_diffterm = np.array(k_diffterm)
        # k_diffterm = k_diffterm.T

        k_joints = jac_1.T @ np.diag(K_muscle) @ jac_1
        k_joints = k_joints #+ k_diffterm
        return k_joints, jac_1, muscle_forces, K_muscle
    
    def linearize(self):
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        pre_calc_state = np.empty(size, np.float64)
        mujoco.mj_getState(self.model, self.data, pre_calc_state, spec)

        n_steps = 3
        act1 = 0.5
        act2 = 0.2

        for m_id in self.muscle_ids:
            self.data.ctrl[m_id] = act1

        for _ in range(n_steps):
            mujoco.mj_step(self.model, self.data)

        a1 = np.array([self.data.act[m_id] for m_id in self.muscle_ids])
        f1 = np.array([self.data.actuator_force[m_id] for m_id in self.muscle_ids])

        mujoco.mj_setState(self.model, self.data, pre_calc_state, spec)

        for m_id in self.muscle_ids:
            self.data.ctrl[m_id] = act2

        for _ in range(n_steps):
            mujoco.mj_step(self.model, self.data)

        a2 = np.array([self.data.act[m_id] for m_id in self.muscle_ids])
        f2 = np.array([self.data.actuator_force[m_id] for m_id in self.muscle_ids])

        m = (f2 - f1)/(a2 - a1)
        M = np.diag(m)
        c = f1 - M @ a1
        return M,c

    def optimize_fast(self, req_torques):
        # provide req_torques in the order: elv_angle, shoulder_elv, rot, elbow 
        # elv_angle is the angle of the elevation plane, and shoulder_elv is the angle in the elevation plane    
        spec = mujoco.mjtState.mjSTATE_INTEGRATION
        size = mujoco.mj_stateSize(self.model, spec)
        state0 = np.empty(size, np.float64)
        mujoco.mj_getState(self.model, self.data, state0, spec)

        mujoco.mj_step1(model.model, model.data)

        jac = self.data.ten_J
        jac_1 = []

        for i in range(self.model.na):
            if i in self.muscle_ids:
                jac_1.append([])
                for j in range(self.model.nq):
                    if j in self.joint_id_list:
                        jac_1[-1].append(jac[i + 4, j])       

        jac_1 = np.array(jac_1)

        M, c = self.linearize()

        initial_guess = [self.act[i] for i in self.muscle_ids] #set initial guess to activations from previous step 

        mujoco.mj_setState(self.model, self.data, state0, spec)

        def cost_fn(activations):
            return np.sum(activations**2)
        
        A = jac_1.T @ M
        C = jac_1.T @ c

        def constraint_fun(activations):
            return A @ activations + C - req_torques
        
        def constraint_jac(activations):
            return A
        
        def objective_grad(activations):
            return 2 * activations

        constraints = {
            'type': 'eq',
            'fun': constraint_fun,
            # 'jac': constraint_jac,
            # 'tol': 1e-3
        }

        bounds = [(0, 1)] * len(self.muscle_ids)

        start_t = time.time_ns()
        res = scipy.optimize.minimize(
            fun=cost_fn,
            x0=initial_guess,
            method='SLSQP',
            constraints=[constraints],
            bounds=bounds,
            options={'disp': False, 'maxiter': 100},
            # jac= objective_grad
        )
        end_t = time.time_ns()
        print(f"Optimization took {(end_t - start_t) / 1e6} ms")

        if not res.success:
            print("Optimization failed:", res.message)

        elv_angle_torq = self.data.qfrc_actuator[self.elv_angle_id] 
        shoulder_elv_torq = self.data.qfrc_actuator[self.shoulder_elv_id] 
        shoulder_rot_torq = self.data.qfrc_actuator[self.rot_id] 
        elbow_torq = self.data.qfrc_actuator[self.elbow_id]

        torq = [elv_angle_torq[0], shoulder_elv_torq[0], shoulder_rot_torq[0], elbow_torq[0]]

        return res.x, res.fun, res.success, end_t - start_t, torq

model_path = "myo_sim/arm/myoarm.xml" 
model = MuscleModel(
    scale=1.0,
    wrist_on=False,
    model_path=model_path,
)


In [3]:
spec = mujoco.mjtState.mjSTATE_INTEGRATION
size = mujoco.mj_stateSize(model.model, spec)
state1 = np.empty(size, np.float64)
mujoco.mj_getState(model.model, model.data, state1, spec)


In [68]:
def find_stiffness(q):
    spec = mujoco.mjtState.mjSTATE_INTEGRATION
    size = mujoco.mj_stateSize(model.model, spec) 
    mujoco.mj_setState(model.model, model.data, state1, spec)
    
    torque = [2,-10,-1,2]
    torque = np.array(torque)

    for i in range(len(model.joint_names)):
        model.data.qpos[model.model.joint(model.joint_names[i]).qposadr] = q[i]
        model.data.qvel[model.model.joint(model.joint_names[i]).qposadr] = 0
        model.data.qacc[model.model.joint(model.joint_names[i]).qposadr] = 0

    mujoco.mj_forward(model.model, model.data)

    spec = mujoco.mjtState.mjSTATE_INTEGRATION
    size = mujoco.mj_stateSize(model.model, spec)
    state0 = np.empty(size, np.float64)
    
    mujoco.mj_getState(model.model, model.data, state0, spec)

    mujoco.mj_inverse(model.model, model.data)
    
    torque = np.array([model.data.qfrc_inverse[id][0] for id in model.joint_id_list])
    torque[0] = -2
    torque[3] = 2
        
    activations, err, success, t, achieved_torque = model.optimize(req_torques=torque)
    k, jac, fm, km = model.stiffness()
    print("Stiffness: ", [k[0,0],k[1,1],k[2,2],k[3,3]])
    print("Achieved torque: ", achieved_torque)
    print("Reqd torque = ", torque)

    # k = np.array([[2.22612592e+01 -1.03751631e+01 -4.63603935e+00  7.32344336e-01],[-1.03751631e+01  5.75668051e+01  2.50795380e+00  2.96334339e-01],[-4.63603935e+00  2.50795380e+00  4.28175375e+00  8.00523337e-03],[ 7.32344336e-01  2.96334339e-01  8.00523337e-03  1.00387523e+01]])
    # k = np.array([[22, -10.3, -4.6, 0.73], [-10.3, 57, 2.5, 0.296], [-4.63, 2.5, 4.28, 0.008], [0.73, 0.296, 0.008, 5*10]])
    mujoco.mj_setState(model.model, model.data, state0, spec)

    ctrl = model.data.ctrl
                
    for idx, act in zip(model.muscle_ids, activations):
        ctrl[idx] = act

    model.data.ctrl = ctrl

    body_name = "lunate"  
    body_id = model.model.body(name=body_name).id

    jacp = np.zeros((3, model.model.nv))  
    jacr = np.zeros((3, model.model.nv))  

    mujoco.mj_jacBodyCom(model.model, model.data, jacp, jacr, body_id)

    J = jacp
    # J = np.vstack((jacp, jacr)) 
    J = J[:, model.joint_id_list]
    J = J[:,:,0]
    
    J_plus = J.T @ np.linalg.inv(J @ J.T)

    K_end = J_plus.T @ k @ J_plus
    K_end = K_end[:3,:3]
    return K_end, k, activations, success, jac, fm, km

In [5]:
sh_arr = [0.5, 0.7, 1.0, 1.2, 1.4, 1.6]
el_arr = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2]

q_arr = []
for i in sh_arr:
    for j in el_arr:
        q_arr.append([i, 1.57, 0.2, j])

In [69]:
data = {'Kq': [], 'Ke': [], 'Km': [], 'el_pos': [], 'wrist_pos': [], 'error': [], 'act': [], 'Jm': [], 'muscle_forces': []}

with mujoco.viewer.launch_passive(model.model, model.data) as viewer:
    viewer.opt.frame = mujoco.mjtFrame.mjFRAME_WORLD

    # for _ in range(100):
    #     mujoco.mj_step(model.model, model.data)
    for i in range(len(q_arr)):
        # try:
        k, k_q, act, err, Jm, fm, km = find_stiffness(q_arr[i])
        # except:
        #     print('error')
        viewer.sync()
        
        body_name = "lunate"  
        body_id = model.model.body(name=body_name).id
        wrist_pos = model.data.xpos[body_id]
        
        elbow_site = "TRI_site_BRA_side"
        site_id = mujoco.mj_name2id(model.model, mujoco.mjtObj.mjOBJ_SITE, elbow_site)
        elbow_pos = model.data.site_xpos[site_id]

        data['Kq'].append(k_q)
        data['Ke'].append(k)
        data['el_pos'].append([elbow_pos[0], elbow_pos[1]])
        data['wrist_pos'].append([wrist_pos[0], wrist_pos[1]])
        data['error'].append(err)
        data['act'].append(act)
        data['Jm'].append(Jm)
        data['muscle_forces'].append(fm)
        data['Km'].append(km)

Optimization took 39.298062 ms
[[-5.28341461e-02 -3.41580148e-03  1.56495813e-03  0.00000000e+00]
 [-2.79922799e-04 -3.00768281e-02  1.40907117e-03  0.00000000e+00]
 [ 3.33884059e-02 -3.62710499e-03  6.04599859e-03  0.00000000e+00]
 [ 1.23350333e-03  2.45394594e-02  9.75690907e-03  0.00000000e+00]
 [ 1.33541233e-03 -1.97852915e-02  1.31688322e-02  0.00000000e+00]
 [-1.25410254e-02  1.36300885e-02 -3.69760103e-03  0.00000000e+00]
 [ 2.66452222e-03 -1.08773115e-02  6.82032297e-03  0.00000000e+00]
 [ 1.82529615e-02  3.21595866e-02  3.23437840e-04  0.00000000e+00]
 [-2.09920928e-02 -3.43555255e-03 -1.29767270e-04  0.00000000e+00]
 [-3.14098184e-02  1.90617267e-02 -9.63777234e-03  0.00000000e+00]
 [-3.05738249e-02  3.37160185e-02 -8.65294513e-03  0.00000000e+00]
 [ 2.23025072e-02  2.35756089e-02 -1.49465713e-03  0.00000000e+00]
 [ 2.20191008e-02  2.96713064e-02  3.35826312e-03  0.00000000e+00]
 [-2.26561091e-02  3.32298964e-02 -1.36154061e-03  0.00000000e+00]
 [ 3.27400658e-02  1.34852099e-

In [70]:
pio.renderers.default = 'browser'

fig = go.Figure()
res = 30
t = np.linspace(0, 2*np.pi, res)
x = np.cos(t)
y = np.sin(t)       
circle = np.vstack((x,y))

for i in range(len(data['Ke'])):
    if data['error'][i]:
        k_e = data['Ke'][i]
        eigvals, eigvecs = np.linalg.eigh(k_e[:2, :2])
        ellipsoid = eigvecs @ np.diag(eigvals) @ circle
        ellipsoid[0,:] = (ellipsoid[0,:])/5e4 + data['wrist_pos'][i][0]
        ellipsoid[1,:] = (ellipsoid[1,:])/5e4 + data['wrist_pos'][i][1]
        
        fig.add_trace(go.Scatter(x=ellipsoid[0,:], y=ellipsoid[1,:]))
        fig.add_trace(go.Scatter(x = [0, data['el_pos'][i][0]], y = [0, data['el_pos'][i][1]], mode='lines', line=dict(color='rgba(0,255,0,0.3)', width=0.5)))
        fig.add_trace(go.Scatter(x = [data['el_pos'][i][0], data['wrist_pos'][i][0]], y = [data['el_pos'][i][1], data['wrist_pos'][i][1]], mode='lines', line=dict(color='rgba(0,0,255,0.3)', width=0.5)))

fig.update_layout(
    width= 800,
    height = 800,
    title = "Endpoint ellipses",
    xaxis=dict(scaleanchor = 'y', range = [-0.8, 0.2]),
    yaxis=dict(range = [-0.8, 0.2]),
)

fig.show()


In [71]:
# Create subplots: each chart in its own row
fig2 = make_subplots(rows=len(data['muscle_forces']), cols=3)

# Add bar charts to each row
for i in range(len(data['muscle_forces'])):
    if data['error'][i]:
        fig2.add_trace(
            go.Bar(x=model.muscle_names, y=data['muscle_forces'][i]),
            row=i+1, col=1
        )
        fig2.add_trace(
            go.Bar(x=model.muscle_names, y=data['act'][i]),
            row=i+1, col=2
        )
        fig2.add_trace(
            go.Bar(x = model.muscle_names, y = data['Km'][i]),
            row = i+1, col = 3
        )

# annotations = []
# for i in range(len(data['muscle_forces'])):
#     annotations.append(dict(
#         x = 0,
#         y = 1 - (i + 0.5)/len(data['muscle_forces']),
#         text = f"Position: {q_arr[i]}, Error: {data['error'][i]}",
#     ))
# Layout adjustments
fig2.update_layout(
    height=100 * len(data['muscle_forces']),  # Height scales with number of charts
    title="Forces and activations",
    showlegend=False,
    margin = dict(l = 100)
)

fig2.show()

Test for muscle activation finding

In [19]:
model.model.opt.timestep = 0.005
print(model.model.opt.timestep)

0.005


In [20]:
# q = [1.59, 0.5, 0.1, 0.8]
q = [0,0,0,0]

for i in range(len(model.joint_names)):
    model.data.qpos[model.model.joint(model.joint_names[i]).qposadr] = q[i]
    model.data.qvel[model.model.joint(model.joint_names[i]).qposadr] = 0
    model.data.qacc[model.model.joint(model.joint_names[i]).qposadr] = 0

In [21]:
spec = mujoco.mjtState.mjSTATE_INTEGRATION
size = mujoco.mj_stateSize(model.model, spec)
state2 = np.empty(size, np.float64)
mujoco.mj_getState(model.model, model.data, state2, spec)

In [22]:
n = 2
lin_arr = []

mujoco.mj_setState(model.model, model.data, state2, spec)

for m_id in model.muscle_ids:
    model.data.ctrl[m_id] = 0.5

for _ in range(n):
    mujoco.mj_step(model.model, model.data)

jac = model.data.ten_J
jac_1 = []
# joint_id_list = [model.shoulder_elv_id, model.elv_angle_id, model.rot_id, model.elbow_id]

for i in range(model.model.na):
    if i in model.muscle_ids:
        jac_1.append([])
        for j in range(model.model.nq):
            if j in model.joint_id_list:
                jac_1[-1].append(jac[i + 4, j])

jac_1 = np.array(jac_1)

a1 = np.array([model.data.act[m_id] for m_id in model.muscle_ids])
f1 = np.array([model.data.actuator_force[m_id] for m_id in model.muscle_ids])
tau1 = np.array([model.data.qfrc_actuator[j_id] for j_id in model.joint_id_list])

mujoco.mj_setState(model.model, model.data, state2, spec)

for m_id in model.muscle_ids:
    model.data.ctrl[m_id] = 0.2

for _ in range(n):
    mujoco.mj_step(model.model, model.data)

a2 = np.array([model.data.act[m_id] for m_id in model.muscle_ids])
f2 = np.array([model.data.actuator_force[m_id] for m_id in model.muscle_ids])
tau2 = np.array([model.data.qfrc_actuator[j_id] for j_id in model.joint_id_list])

m = (f2 - f1)/(a2 - a1)
M = np.diag(m)
c = f1 - M @ a1

# print(M, c)

print("f2: ", f2)
print("Calc f2: ", M @ a2 + c)

print("f1: ", f1)
print("Calc f1: ", M @ a1 + c)

mujoco.mj_setState(model.model, model.data, state2, spec)

for m_id in model.muscle_ids:
    model.data.ctrl[m_id] = 0.05

for _ in range(n):
    mujoco.mj_step(model.model, model.data)

a3 = np.array([model.data.act[m_id] for m_id in model.muscle_ids])
f3 = np.array([model.data.actuator_force[m_id] for m_id in model.muscle_ids])
tau3 = np.array([model.data.qfrc_actuator[j_id] for j_id in model.joint_id_list])

print("f3: ", f3)
print("Calc f3: ", M @ a3 + c)

print("tau1: ", tau1)
print("Calc tau1: ", jac_1.T @ M @ a1 + jac_1.T @ c)

f2:  [-255.19047972 -680.93776683  -32.77040651  -91.42981088 -439.4630386
 -222.29940681 -107.19127079  -44.40057069  -88.97313225  -34.565842
   -0.87867933 -106.15653028  -11.9343214   -24.86817146 -132.90322348
  -79.73514078  -53.64659322   -1.07868991  -70.10043454 -167.74535059
  -55.86043207 -381.95647876 -166.21556637]
Calc f2:  [-255.19047972 -680.93776683  -32.77040651  -91.42981088 -439.4630386
 -222.29940681 -107.19127079  -44.40057069  -88.97313225  -34.565842
   -0.87867933 -106.15653028  -11.9343214   -24.86817146 -132.90322348
  -79.73514078  -53.64659322   -1.07868991  -70.10043454 -167.74535059
  -55.86043207 -381.95647876 -166.21556637]
f1:  [ -527.36503493  -859.33636219   -81.92601628  -225.77829773
 -1098.6575965   -555.74851702  -267.97817696  -111.00142673
  -180.45849414   -86.41460499    -2.19669834  -265.3913257
   -29.83580349   -62.17042865  -332.2580587   -199.33785194
  -134.11648306    -2.69672477  -171.55586484  -265.77545026
  -139.6468339   -665.8751

In [27]:
print(M,c)

[[ -907.24851737     0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.        ]
 [    0.          -594.66198453     0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.        ]
 [    0.             0.          -163.85203257     0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.        ]
 [    0.             0.             0.        

In [23]:
np.set_printoptions(threshold=np.inf)
print(model.data.ten_J[20])
print(model.elv_angle_id, model.shoulder_elv_id, model.rot_id, model.elbow_id)

[ 1.54498810e-18  1.75640752e-17  1.75640752e-17  6.77626358e-19
 -5.20417043e-18  8.67361738e-19  3.03576608e-18 -4.98732999e-18
 -1.88109077e-17  1.02999206e-18  3.18484388e-18  1.53956708e-17
 -5.42101086e-20 -7.58941521e-19  2.01334493e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
[10] [11] [13] [14]


In [24]:
for i, n in zip(model.muscle_ids, model.muscle_names):
    print(i, n)

0 DELT1
1 DELT2
2 DELT3
3 SUPSP
4 INFSP
5 SUBSC
6 TMIN
7 TMAJ
8 PECM1
9 PECM2
10 PECM3
12 LAT2
13 LAT3
14 CORB
15 TRIlong
16 TRIlat
17 TRImed
18 ANC
19 SUP
20 BIClong
21 BICshort
22 BRA
23 BRD


In [25]:
mujoco.mj_setState(model.model, model.data, state2, spec)

model.data.qpos[model.elbow_id] = 0.0

mujoco.mj_step1(model.model, model.data)
mujoco.mj_fwdActuation(model.model, model.data)

j_id = model.elbow_id
print(model.data.qfrc_actuator[j_id])
# print(model.data.qpos)

mujoco.mj_setState(model.model, model.data, state2, spec)
model.data.qpos[model.elbow_id] = 1.0
# model.data.ctrl[20] = 0.2
mujoco.mj_step1(model.model, model.data)

print(model.data.actuator_force)
mujoco.mj_fwdActuation(model.model, model.data)

print(model.data.qfrc_actuator)

# print(model.data.actuator_force)
print("Force calc")
print(model.data.ten_J[4:][:].T @ model.data.actuator_force)
# print(model.data.qfrc_actuator)

[3.93841156]
[-7.37865736e+01 -5.60651538e+02  0.00000000e+00 -1.99421882e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.80178485e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.43957237e+00
 -1.02699488e+02 -4.86569755e-04 -1.93229162e+02 -1.42229717e+02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.28633947e-02
  0.00000000e+00  0.00000000e+00 -1.02661574e-01  0.00000000e+00
  0.00000000e+00 -1.86879814e-01  0.00000000e+00  0.00000000e+00
  0.00000000

In [26]:
for i in range(model.model.ntendon):
    print(mujoco.mj_id2name(model.model, mujoco.mjtObj.mjOBJ_TENDON, i))

s_glenohum_ligament
m_glenohum_ligament
i_glenohum_ligament
coracohum_ligament
DELT1_tendon
DELT2_tendon
DELT3_tendon
SUPSP_tendon
INFSP_tendon
SUBSC_tendon
TMIN_tendon
TMAJ_tendon
PECM1_tendon
PECM2_tendon
PECM3_tendon
LAT1_tendon
LAT2_tendon
LAT3_tendon
CORB_tendon
TRIlong_tendon
TRIlat_tendon
TRImed_tendon
ANC_tendon
SUP_tendon
BIClong_tendon
BICshort_tendon
BRA_tendon
BRD_tendon
ECRL_tendon
ECRB_tendon
ECU_tendon
FCR_tendon
FCU_tendon
PL_tendon
PT_tendon
PQ_tendon
FDS5_tendon
FDS4_tendon
FDS3_tendon
FDS2_tendon
FDP5_tendon
FDP4_tendon
FDP3_tendon
FDP2_tendon
EDC5_tendon
EDC4_tendon
EDC3_tendon
EDC2_tendon
EDM_tendon
EIP_tendon
EPL_tendon
EPB_tendon
FPL_tendon
APL_tendon
OP_tendon
RI2_tendon
LU_RB2_tendon
UI_UB2_tendon
RI3_tendon
LU_RB3_tendon
UI_UB3_tendon
RI4_tendon
LU_RB4_tendon
UI_UB4_tendon
RI5_tendon
LU_RB5_tendon
UI_UB5_tendon
